In [1]:
import pandas as pd
import numpy as np
import re
import os

pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [ ]:
pdfs = pd.read_csv('web-pdf.csv')
pdfs.head()

In [3]:
pdfs = pdfs.rename(columns={'contacto': 'nombre', 
                            'Equipment': 'amueblado', 
                            'precio_alquiler': 'precio',
                            'teléfono': 'telefono',
                            'baños': 'banos',
                            })
pdfs['tipo_transaccion'] = 'Alquiler'
pdfs['perfil'] = 'Particular'
pdfs['fecha'] = '2018-01-01'
pdfs['precio'] = pdfs['precio'].fillna(0).astype(int)
pdfs['superficie'] = pdfs['superficie'].fillna(0).astype(int)
pdfs['banos'] = pdfs['banos'].fillna(0).astype(int)
# Cambiar datos concretos de una columna, por ejemplo, cambiar el valor de 'precio' donde 'ciudad' es 'Murcia'
# Reemplazar los valores de 'No especificado' en la columna 'habitaciones' por 0
pdfs.loc[pdfs['habitaciones'] == 'No especificado', 'habitaciones'] = 0
# Cambiar el tipo de la columna 'habitaciones' a entero
pdfs['habitaciones'] = pdfs['habitaciones'].astype(int)



# Definir la función para extraer los datos
def extraer_datos(row):
    # Usar una expresión regular para capturar "tipo de vivienda", "street" y "place"
    match = re.search(r'Alquiler de (.+?) en (.*?, \d+), (.*?)\.pdf', row)
    if match:
        tipo = match.group(1)  # Captura el tipo de vivienda (piso, casa, etc.)
        calle = match.group(2)         # Captura la calle y número
        barrio = match.group(3)          # Captura el barrio y la ciudad
        return pd.Series([tipo, calle, barrio])
    else:
        return pd.Series([None, None, None])  # Si no coincide, devuelve None

# Aplicar la función al DataFrame y crear las nuevas columnas
pdfs[['tipo', 'calle', 'barrio']] = pdfs['pdf'].apply(extraer_datos)

In [2]:
particulares = pd.read_csv('../../soporte/particulares/particulares_murcia.csv')
particulares= particulares.drop(['Unnamed: 0'], axis=1)
particulares = particulares.rename(columns={'date_scrape':'fecha'})
# Separar la información de la columna 'barrio' en dos columnas: 'barrio' y 'ciudad'
# Utiliza 'expand=True' y luego 'n=1' para asegurarse de que la división tenga dos partes (rellenará con NaN si no hay coma)
particulares[['barrio', 'ciudad']] = particulares['barrio'].str.split(',', n=1, expand=True)

# Limpiar espacios en ambas columnas
particulares['barrio'] = particulares['barrio'].str.strip()
particulares['ciudad'] = particulares['ciudad'].str.strip().fillna('Murcia')

# Unir las columnas 'calle', 'barrio' y 'ciudad' para crear la nueva columna 'localizacion'
particulares['localizacion'] = particulares['calle'] + ', ' + particulares['barrio'] + ', ' + particulares['ciudad']

# Eliminar filas donde la columna 'telefono' esté vacía
particulares = particulares.dropna(subset=['telefono'])

particulares['fuente'] = 'pdf'

particulares.head(5) 

,nombre,telefono,tipo_transaccion,tipo,tipo_contacto,calle,barrio,superficie,habitaciones,planta,caracteristicas,precio,id_inmueble,fecha,banos,estado,armarios,orientacion,amueblado,exterior_interior,ascensor,ciudad,localizacion,fuente
0,Antonio,633 860 715,Alquiler,Piso,Particular,"calle Valle de Leiva, 5",Puente Tocinos,110,3,1,"110 m² construidos, 90 m² útiles 3 habitaciones 2 baños Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",495,26861942,2019-11-29,2,Segunda mano/buen estado,Sí,sur,Sí,Exterior,Sí,Murcia,"calle Valle de Leiva, 5, Puente Tocinos, Murcia",pdf
1,pascual,968 264 704,Alquiler,Piso,Particular,"calle Escuelas, 41",Alquerías,116,3,1,116 m² construidos 3 habitaciones 2 baños Balcón Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción individual Certificación energética: (IPE no indicado) Vivienda apta para su uso por personas con movilidad reducida,490,83207117,2019-11-29,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escuelas, 41, Alquerías, Murcia",pdf
2,Ender Fuenmayor,627 226 825,Alquiler,Piso,Particular,"calle Escritor Francisco Aleman Sainz, 2",Espinardo,95,3,5,"95 m² construidos, 73 m² útiles 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción central Certificación energética: no indicado Acceso adaptado a personas con movilidad reducida Vivienda apta para su uso por personas con movilidad reducida",500,83714401,2022-06-14,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escritor Francisco Aleman Sainz, 2, Espinardo, Murcia",pdf
3,Alberto,659 772 242,Alquiler,Piso,Particular,"carril Cuevas, 14",Cabezo de Torres,100,3,1,100 m² construidos 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Trastero Orientación este Cocina equipada y casa sin amueblar Calefacción individual: Bomba de frío/calor Certificación energética: no indicado,500,83714094,2019-11-29,2,Segunda mano/buen estado,No,este,No,Exterior,Sí,Murcia,"carril Cuevas, 14, Cabezo de Torres, Murcia",pdf
4,Timoteo Gimenez,606 069 412,Alquiler,Ático,Particular,"calle Jardines, 1",Alhama de Murcia,90,2,0,"90 m² construidos, 80 m² útiles 2 habitaciones 1 baño Terraza Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",350,1675052,2019-11-29,1,Segunda mano/buen estado,Sí,sur,Sí,Exterior,No,Murcia,"calle Jardines, 1, Alhama de Murcia, Murcia",pdf


In [3]:
# Leer archivo CSV
web = pd.read_csv("../../web_2/data/inmuebles.csv")

# Eliminar columnas innecesarias
web = web.drop(['url', 'superficie_util', 'referencia'], axis=1)

# Renombrar columnas
web = web.rename(columns={'anunciante': 'tipo_contacto', 'tlf': 'telefono', 'tipo': 'tipo_transaccion', 'id': 'id_inmueble'})

# Limpiar y convertir la columna 'precio'
web['precio'] = web['precio'].str.replace('.', '').str.replace('€', '').str.strip()
web['precio'] = pd.to_numeric(web['precio'], errors='coerce').fillna(0).astype(int)

# Limpiar la columna 'precio_metro'
web['precio_metro'] = web['precio_metro'].str.replace(',', '.').str.strip().astype(float)

# Convertir las columnas 'habitaciones', 'baños', y 'construccion' a numéricas
web['habitaciones'] = pd.to_numeric(web['habitaciones'], errors='coerce').fillna(0).astype(int)
web["baños"] = pd.to_numeric(web['baños'], errors='coerce').fillna(0).astype(int)
# web["construccion"] = pd.to_numeric(web['construccion'], errors='coerce').fillna(0).astype(int)

# Extraer información de la columna 'caracteristicas'

# Extraer número de baños (int)
web['banos'] = web['caracteristicas'].str.extract(r'(\d+) baños?', expand=False).astype('float').fillna(0).astype('int')

# Extraer estado del inmueble
web['estado'] = web['caracteristicas'].str.extract(r'(Segunda mano\/buen estado|Nueva construcción|A reformar)', expand=False).fillna('No indicado')

# Extraer si tiene armarios empotrados (Sí o No)
web['armarios'] = web['caracteristicas'].str.contains('Armarios empotrados', case=False, na=False).map({True: 'Sí', False: 'No'})

# Extraer orientación (norte, sur, este, oeste)
web['orientacion'] = web['caracteristicas'].str.extract(r'Orientación (\w+)', expand=False).fillna('No indicado')

# Extraer si está amueblado (Sí o No)
web['amueblado'] = web['caracteristicas'].str.contains('Amueblado', case=False, na=False).map({True: 'Sí', False: 'No'})

# Extraer si es exterior o interior
web['exterior_interior'] = web['caracteristicas'].str.extract(r'(exterior|interior)', expand=False).fillna('No indicado').str.capitalize()

# Extraer si tiene ascensor (Sí o No)
web['ascensor'] = web['caracteristicas'].str.contains('con ascensor', case=False, na=False).map({True: 'Sí', False: 'No'})

# Quitar el prefijo '34' si el número empieza con este prefijo
# web['telefono'] = web['telefono'].str.replace(r'^(\+34|0034|34)\s*', '', regex=True)

# Eliminar filas donde la columna 'telefono' esté vacía después del proceso
web = web.dropna(subset=['telefono'])

# Extraer solo la parte de la fecha (sin la hora)
web['fecha'] = web['fecha'].str.split(' ', n=1, expand=True)[0]

web['fuente']= 'idealista'

# Mostrar las primeras 5 filas del DataFrame actualizado
web.head(5)


,id_inmueble,tipo_transaccion,titulo,calle,barrio,zona,ciudad,localizacion,precio,precio_metro,caracteristicas,habitaciones,baños,tipo_contacto,nombre,telefono,fecha,superficie,terraza,garaje,estado,armarios,trastero,orientacion,calefaccion,planta,ascensor,construccion,banos,amueblado,exterior_interior,fuente
0,106619687,Ventas,Chalet pareado en venta en calle San Antonio,Calle San Antonio,Beniaján,Este,Murcia,"Calle San Antonio, Barrio Beniaján, Distrito Pedanías Este, Murcia, Área de Murcia, Murcia",159900,457.000,"['Chalet pareado', '2 plantas', '350 m² construidos, 329 m² útiles', '5 habitaciones', '3 baños', 'Parcela de 311 m²', 'Terraza y balcón', 'Segunda mano/para reformar', 'Trastero', 'Construido en 1969']",5,3,Profesional,INMOAMM *,34868350946,2024-11-21,350,Terraza,NaN,No indicado,No,Trastero,No indicado,NaN,NaN,No,1969.0,3,No,No indicado,idealista
1,106621017,Ventas,Casa o chalet independiente en venta en Algezares,NaN,Algezares,Este,Murcia,"Barrio Algezares, Distrito Pedanías Este, Murcia, Área de Murcia, Murcia",259000,1.727,"['Casa o chalet independiente', '1 planta', '150 m² construidos, 140 m² útiles', '3 habitaciones', '2 baños', 'Parcela de 1.200 m²', 'Terraza', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Armarios empotrados', 'Orientación sur', 'Calefacción central', 'Solo interior de la vivienda adaptado para personas con movilidad reducida']",3,2,Profesional,I Love Murcia servicios inmobiliarios,34868350397,2024-11-21,150,Terraza,Plaza de garaje,Segunda mano/buen estado,Sí,NaN,sur,central,NaN,No,NaN,2,No,Interior,idealista
2,106621744,Ventas,Casa o chalet independiente en venta en La Albatalía,NaN,La Albatalía,Este,Murcia,"Barrio La Albatalía, Distrito Pedanías Oeste, Murcia, Área de Murcia, Murcia",380000,1.490,"['Casa o chalet independiente', '1 planta', '255 m² construidos, 215 m² útiles', '5 habitaciones', '2 baños', 'Parcela de 1.816 m²', 'Terraza', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Armarios empotrados', 'Trastero', 'Orientación sur', 'Construido en 1990', 'Calefacción central']",5,2,Profesional,Inmobiliaria Pujante,34868350423,2024-11-21,255,Terraza,Plaza de garaje,Segunda mano/buen estado,Sí,Trastero,sur,central,NaN,No,1990.0,2,No,No indicado,idealista
3,106623206,Ventas,Finca rústica en venta en Zeneta,NaN,Zeneta,Este,Murcia,"Barrio Zeneta, Distrito Pedanías Este, Murcia, Área de Murcia, Murcia",80000,468.000,"['Finca rústica', '1 planta', '171 m² construidos', '1 habitación', 'Sin baños', 'Parcela de 2.296 m²', 'Segunda mano/para reformar', 'Orientación sur', 'No dispone de calefacción']",0,0,Profesional,ALQUILOTUCASA MURCIA1,34868350971,2024-11-21,171,NaN,NaN,No indicado,No,NaN,sur,NaN,NaN,No,NaN,0,No,No indicado,idealista
4,106622750,Ventas,Casa o chalet independiente en venta en Churra,NaN,Churra,Norte,Murcia,"Barrio Churra, Distrito Pedanías Norte, Murcia, Área de Murcia, Murcia",325000,1.609,"['Casa o chalet independiente', '1 planta', '202 m² construidos, 169 m² útiles', '4 habitaciones', '3 baños', 'Terraza', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Trastero', 'Orientación norte, sur', 'Construido en 2006', 'No dispone de calefacción']",4,3,Profesional,Expoviviendas,34868350855,2024-11-21,202,Terraza,Plaza de garaje,Segunda mano/buen estado,No,Trastero,norte,NaN,NaN,No,2006.0,3,No,No indicado,idealista


In [4]:
web['tipo_transaccion'].value_counts()

tipo_transaccion
Ventas      5740
Alquiler    1284
Name: count, dtype: int64

In [5]:
web['tipo_contacto'].value_counts()

tipo_contacto
Profesional    5457
Particular      972
Promotora       162
Agente Pro       16
Name: count, dtype: int64

In [6]:
# Unir los DataFrames 'web' y 'particulares' utilizando 'id_inmueble' como clave
total_web_pdf = pd.concat([particulares, web], ignore_index=True)
total_web_pdf

,nombre,telefono,tipo_transaccion,tipo,tipo_contacto,calle,barrio,superficie,habitaciones,planta,caracteristicas,precio,id_inmueble,fecha,banos,estado,armarios,orientacion,amueblado,exterior_interior,ascensor,ciudad,localizacion,fuente,titulo,zona,precio_metro,baños,terraza,garaje,trastero,calefaccion,construccion
0,Antonio,633 860 715,Alquiler,Piso,Particular,"calle Valle de Leiva, 5",Puente Tocinos,110,3,1.0,"110 m² construidos, 90 m² útiles 3 habitaciones 2 baños Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",495,26861942,2019-11-29,2,Segunda mano/buen estado,Sí,sur,Sí,Exterior,Sí,Murcia,"calle Valle de Leiva, 5, Puente Tocinos, Murcia",pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pascual,968 264 704,Alquiler,Piso,Particular,"calle Escuelas, 41",Alquerías,116,3,1.0,116 m² construidos 3 habitaciones 2 baños Balcón Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción individual Certificación energética: (IPE no indicado) Vivienda apta para su uso por personas con movilidad reducida,490,83207117,2019-11-29,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escuelas, 41, Alquerías, Murcia",pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ender Fuenmayor,627 226 825,Alquiler,Piso,Particular,"calle Escritor Francisco Aleman Sainz, 2",Espinardo,95,3,5.0,"95 m² construidos, 73 m² útiles 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Cocina equipada y casa sin amueblar Calefacción central Certificación energética: no indicado Acceso adaptado a personas con movilidad reducida Vivienda apta para su uso por personas con movilidad reducida",500,83714401,2022-06-14,2,Segunda mano/buen estado,Sí,No indicado,No,Exterior,Sí,Murcia,"calle Escritor Francisco Aleman Sainz, 2, Espinardo, Murcia",pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alberto,659 772 242,Alquiler,Piso,Particular,"carril Cuevas, 14",Cabezo de Torres,100,3,1.0,100 m² construidos 3 habitaciones 2 baños Plaza de garaje incluida en el precio Segunda mano/buen estado Trastero Orientación este Cocina equipada y casa sin amueblar Calefacción individual: Bomba de frío/calor Certificación energética: no indicado,500,83714094,2019-11-29,2,Segunda mano/buen estado,No,este,No,Exterior,Sí,Murcia,"carril Cuevas, 14, Cabezo de Torres, Murcia",pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Timoteo Gimenez,606 069 412,Alquiler,Ático,Particular,"calle Jardines, 1",Alhama de Murcia,90,2,0.0,"90 m² construidos, 80 m² útiles 2 habitaciones 1 baño Terraza Plaza de garaje incluida en el precio Segunda mano/buen estado Armarios empotrados Orientación sur Amueblado y cocina equipada Certificación energética: no indicado",350,1675052,2019-11-29,1,Segunda mano/buen estado,Sí,sur,Sí,Exterior,No,Murcia,"calle Jardines, 1, Alhama de Murcia, Murcia",pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8337,The Simple Rent,34868350665,Alquiler,NaN,Profesional,Avenida Don Juan de Borbón,Vista Alegre,70,2,NaN,"['70 m² construidos, 66 m² útiles', '2 habitaciones', '1 baño', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Armarios empotrados', 'Trastero', 'Amueblado y cocina equipada', 'Planta 3ª exterior', 'Con ascensor']",770,1473835,2024-10-18,1,Segunda mano/buen estado,Sí,No indicado,Sí,Exterior,Sí,Murcia,"Avenida Don Juan de Borbón, Barrio Vista Alegre, Distrito Norte, Murcia, Área de Murcia, Murcia",idealista,Piso en avenida Don Juan de Borbón,Norte,11.00,1.0,NaN,Plaza de garaje,Trastero,NaN,NaN
8338,AM,34676155485,Alquiler,NaN,Particular,"Calle Fernando Sanchéz Agüera, 28",El Palmar,57,0,NaN,"['57 m² construidos', '1 habitación', '1 baño', 'Terraza y balcón', 'Plaza de garaje incluida en el precio', 'Segunda mano/buen estado', 'Amueblado y coci

In [7]:
total_web_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8342 entries, 0 to 8341
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   nombre             7960 non-null   object 
 1   telefono           8342 non-null   object 
 2   tipo_transaccion   8342 non-null   object 
 3   tipo               1318 non-null   object 
 4   tipo_contacto      7925 non-null   object 
 5   calle              5235 non-null   object 
 6   barrio             8342 non-null   object 
 7   superficie         8342 non-null   int64  
 8   habitaciones       8342 non-null   object 
 9   planta             1318 non-null   float64
 10  caracteristicas    8267 non-null   object 
 11  precio             8342 non-null   int64  
 12  id_inmueble        8342 non-null   int64  
 13  fecha              8342 non-null   object 
 14  banos              8342 non-null   int64  
 15  estado             8342 non-null   object 
 16  armarios           8342 

In [8]:
total_web_pdf.to_csv('basedatos.csv')